# Pipeline de préparation des données 

## **OBJECTIF GENERAL**

**Etablir un code propre et optimisé de traitement du contenu et métadonnées issus du serveur Erudit**

## **OBJECTIFS SPECIFIQUES**

- Nettoyer le texte 
- Nettoyer les métadonnées
- Filtrer le vocabulaire

## **METHODE**

**I. Nettoyage du texte**

Tokenization : NLTK

Enlever mots outils : **NLTK + Spacy stopwords** : STOPWORDS

Enlever mots trop courts : longueur <= 2

Lemmatizer: **FrenchLeffLemmatizer**

Enlever documents "trop courts" (< 300 mots)

**II. Nettoyage des métadonnées**

Détecter valeurs manquantes pour certains attributs

Gros du travail est sur les auteurs : affiliation à nettoyer et Nom/Prénom pour disambiguation mais pas nécessaire ici

**III. Vocabulaire**

Enlever mots rares/fréquents

Correcteur orthographique et comparaison avec dictionnaire de référence : **algorithme de Norvig** (http://norvig.com/spell-correct.html) et **dictionnaire FR Gutenberg**

## **PIPELINE**

**INPUT** :

Corpus = Liste de dictionnaire contenant le texte et métadonnées d'intérêt de chaque article, prêt à être nettoyé.

Issu de *Extraction_XML_to_CORPUS.ipynb*

**OUTPUT** :

- Corpus_clean = Matrices sac de mots pour chaque revue
- tokens_bigrams_corpus_clean = liste de tokens pour chaque article de chaque revue
- dictionary = dictionnaire filtré pour chaque revue

**Import Bibliothèques**

In [ ]:
import spacy 
import nltk, re
from nltk.corpus import stopwords
from nltk import word_tokenize

import gensim
from gensim.corpora import Dictionary

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

import matplotlib.pyplot as plt
from tqdm import tqdm
import time

# Charge les mots stops en français de NLTK et Spacy
from spacy.lang.fr.stop_words import STOP_WORDS
nltk.download('stopwords')

# Ajout des stopwords de NLTK dans ceux de Spacy
for word in stopwords.words('french'):
    STOP_WORDS.add(word)

In [ ]:
# Récupération d'une liste de dictionnaire  pour chaque revue du type 
# {'texte' : texte article brut , 'metadata': {'title' : titre, 'typeart': , 'lang': , 'annee': ,'info_auteurs': [(prenom, nom, affiliation)]}, 'URL': }
%store -r Corpus_AE
%store -r Corpus_EI
%store -r Corpus_RI

-------------------

# I. Nettoyage du texte

-----------------

### **1) Préprocessing : tokenization et nettoyage avec NLTK**

**OUTPUT DE CETTE PARTIE** 

In [ ]:
% store -r Corpus_LDA_AE
% store -r Corpus_LDA_EI
% store -r Corpus_LDA_RI

%store Corpus_LDA_AE_lemma
%store Corpus_LDA_EI_lemma
%store Corpus_LDA_RI_lemma

In [ ]:
def prepare_corpus_LDA (Corpus, stop = STOP_WORDS, lemma = False):
    """ Traitement de chaque article pour tokeniser les mots de chaque article de type article
    ATTENTION: les indices Corpus_AE et Corpus_AE_LDA ne sont pas les mêmes car on ne garde que les articles de type article
    Se référer à l'URL pour faire la correspondance entre les 2 corpus"""
    
    Corpus_LDA = []
    
    if lemma == True :
        lemmatizer = FrenchLefffLemmatizer()
    
    for index_document in tqdm(range(len(Corpus))) :
    # on ne traite que les articles de type article et en français
        if (Corpus[index_document]['metadata']['typeart'], Corpus[index_document]['metadata']['lang']) == ('article','fr') : 

            # tokenization avec NLTK
            tokens = word_tokenize(Corpus[index_document]['texte']) 

            # Enlever les documents trop courts : moins de 300 mots (justifier heuristique)
            if len(tokens) < 300 :
                pass

            # Enlever les chiffres et convertir en minuscules
            tokens = [token.lower() for token in tokens if token.isalpha()]

            # Enlever les mots de moins de 3 caractères (ponctuation) et les stopwords
            tokens = [token for token in tokens if len(token) > 2 and token not in stop]       

            # Lemmatisation des mots
            if lemma == True :
                tokens = [lemmatizer.lemmatize(token) for token in tokens]

            # Ajout des mots filtrés à la matrice document/mots
            Corpus_LDA.append((Corpus[index_document]['URL'], tokens))

    return Corpus_LDA

**SANS LEMMATISATION**

In [ ]:
# Running time = 2 min par revue
Corpus_LDA_AE = prepare_corpus_LDA(Corpus_AE, lemma=False)
Corpus_LDA_EI = prepare_corpus_LDA(Corpus_EI, lemma = False)
Corpus_LDA_RI = prepare_corpus_LDA(Corpus = Corpus_RI, lemma = False)

In [ ]:
%store Corpus_LDA_AE
%store Corpus_LDA_EI
%store Corpus_LDA_RI

**AVEC LEMMATISATION**

In [ ]:
# Running time = 2 min par corpus
Corpus_LDA_AE_lemma = prepare_corpus_LDA(Corpus_AE, lemma=True)
Corpus_LDA_EI_lemma = prepare_corpus_LDA(Corpus_EI, lemma=True)
Corpus_LDA_RI_lemma = prepare_corpus_LDA(Corpus_RI, lemma=True)

In [ ]:
%store Corpus_LDA_AE_lemma
%store Corpus_LDA_EI_lemma
%store Corpus_LDA_RI_lemma

### **2) Formation du vocabulaire pour le corpus**

a) Récupération des bigrammes

b) Enlever mots rares/fréquents

c) Correcteur orthographique et comparaison avec dictionnaire de référence : **algorithme de Norvig** (https://github.com/barrust/pyspellchecker)

**USAGE**

In [ ]:
%store -r Corpus_LDA_AE
%store -r Corpus_LDA_EI
%store -r Corpus_LDA_RI

In [ ]:
%store -r Corpus_LDA_AE_lemma
%store -r Corpus_LDA_EI_lemma
%store -r Corpus_LDA_RI_lemma

####  a) Bigrammes

In [ ]:
# Ajout des bigrammes
def add_bigrams(docs):
    from gensim.models.phrases import Phraser, Phrases
    phrases = Phrases(docs, min_count=20) # initialisation hyperparamètre = nombre d'occurences d'une paire
    bigram = Phraser(phrases) # wrapper plus efficace
    for idx in tqdm(range(len(docs))):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    return docs

**SANS LEMMATISATION**

In [ ]:
# On conserve une liste de tokens pour chaque article en vue de créer le vocabulaire
tokens_Corpus_LDA_AE = [element[1] for element in Corpus_LDA_AE]
tokens_Corpus_LDA_EI = [element[1] for element in Corpus_LDA_EI]
tokens_Corpus_LDA_RI = [element[1] for element in Corpus_LDA_RI]

In [ ]:
# Running time : 2min par revue
tokens_bigrams_Corpus_LDA_AE= add_bigrams(tokens_Corpus_LDA_AE)
tokens_bigrams_Corpus_LDA_EI= add_bigrams(tokens_Corpus_LDA_EI)
tokens_bigrams_Corpus_LDA_RI= add_bigrams(tokens_Corpus_LDA_RI)

**AVEC LEMMATISATION**

In [ ]:
# On conserve une liste de tokens pour chaque article en vue de créer le vocabulaire
tokens_Corpus_LDA_AE_lemma = [element[1] for element in Corpus_LDA_AE_lemma]
tokens_Corpus_LDA_EI_lemma = [element[1] for element in Corpus_LDA_EI_lemma]
tokens_Corpus_LDA_RI_lemma = [element[1] for element in Corpus_LDA_RI_lemma]

In [ ]:
tokens_bigrams_Corpus_LDA_AE_lemma= add_bigrams(tokens_Corpus_LDA_AE_lemma)
tokens_bigrams_Corpus_LDA_EI_lemma= add_bigrams(tokens_Corpus_LDA_EI_lemma)
tokens_bigrams_Corpus_LDA_RI_lemma= add_bigrams(tokens_Corpus_LDA_RI_lemma)

#### b) Formation du dictionnaire et nettoyage des mots rares/fréquents

### i. TRAITEMENT POUR ACTUALITE ECONOMIQUE

**SANS LEMMATISATION**

In [ ]:
# Création du dictionnaire
dictionary_AE = Dictionary(tokens_bigrams_Corpus_LDA_AE)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE))

**AVEC LEMMATISATION**

In [ ]:
# Création du dictionnaire
dictionary_AE_lemma = Dictionary(tokens_bigrams_Corpus_LDA_AE_lemma)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE_lemma))

**MOTS RARES ET FREQUENTS**

**a) Diversité mots**

**FILTRE ABOVE**

In [ ]:
# Création des dictionnaires selon chaque filtre (à faire une fois pour les 2 stats)
filtres_above = list(range(5,105,5))
dicos_filtres_above_AE = []
for filtre in tqdm(filtres_above):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_AE)
    dico.filter_extremes(no_above=filtre/100)
    dicos_filtres_above_AE.append(dico)

**FILTRE BELOW**

In [ ]:
filtres_below = list(range(1,20)) # filtre below est en nombre absolu de documents
dicos_filtres_below_AE = []
for filtre in tqdm(filtres_below):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_AE)
    dico.filter_extremes(no_below=filtre)
    dicos_filtres_below_AE.append(dico)

**PLOTS**

In [ ]:
def plot_filtre (initiales= 'AE', relatif=True,type_filtre="above"):
    fig = plt.figure(figsize=(10,5))
    
    if type_filtre == "above":
        filtres = filtres_above
        nb_mots_restants= [len(dico) for dico in dicos_filtres_above_AE]
        plt.xlabel('Seuil de filtre des mots fréquents (en pourcentage d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[-1] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.98)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres, nb_mots_restants, color='k')
    
    elif type_filtre == "below": 
        filtres = filtres_below
        nb_mots_restants= [len(dico) for dico in dicos_filtres_below_AE]
        plt.xlabel('Seuil de filtre des mots rares (en nombre d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[0] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.4)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres[0:], nb_mots_restants[0:], color='burlywood')
    
    plt.ylabel('Taille relative du vocabulaire',fontsize=20)
    #plt.title("Diversité du vocabulaire en fonction du filtre des mots appaissant dans plus d'un certain pourcent de documents")
    plt.title(initiales, fontsize=40)
    plt.savefig("Plots/Filtre vocabulaire/filtre_" + type_filtre+"_absolu_AE.png")
    plt.show()

In [ ]:
plot_filtre (initiales= 'AE', relatif=True,type_filtre="below")

**CONCLUSION**

On peut choisir un filtre above à **30% : 90% de diversité des mots est conservée.**

On choisit un filtre below à n = 3 documents, ce qui diminue la diversité du vocabulaire de 50%.

**2) ALGORITHME DE NORVIG ET CORRECTEUR ORTHOGRAPHIQUE**

Idée : utiliser distance de Levenshtein pour trouver les mots du vocabulaire les plus proches d'un mot donné, et donner la correction la plus probable du mot en fonction de sa fréquence générale dans la langue.

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker(language='fr', distance=1)

**SANS LEMMATISATION**

In [ ]:
tokens_filtré_AE = [dictionary_AE[w] for w in dictionary_AE]
misspelled = spell.unknown(tokens_filtré_AE)
corrections = []
nb_bigrams = 0
for token in misspelled:
    if '_' in token: # on ne corrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_AE)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_AE)*100,1), "% des tokens.")

In [ ]:
dico_correction = {}
for element in corrections:
    dico_correction[element[0]]= element[1]

**AVEC LEMMATISATION**

In [ ]:
tokens_filtré_AE_lemma = [dictionary_AE_lemma[w] for w in dictionary_AE_lemma]

In [ ]:
misspelled_lemma = spell.unknown(tokens_filtré_AE_lemma)
corrections = []
nb_bigrams = 0
for token in misspelled_lemma:
    if '_' in token: # on ne corrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_AE_lemma)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_AE_lemma)*100,1), "% des tokens.")

In [ ]:
dico_correction_lemma = {}
for element in corrections:
    dico_correction_lemma[element[0]]= element[1]

**3) FORMATION DU DICTIONNAIRE FINAL**

REMARQUE : il faut appliquer le filtre de correction en 1er en fait, car le dictionnaire gensim ne peut être construit qu'à partir de la représentation liste de liste des documents

**FILTRE CORRECTION**

**SANS LEMMATISATION**

In [ ]:
tokens_bigrams_Corpus_LDA_AE_clean = tokens_bigrams_Corpus_LDA_AE

In [ ]:
for index_element in tqdm(range(len(tokens_bigrams_Corpus_LDA_AE))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_AE[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_AE_clean[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_AE_clean[index_element]]

In [ ]:
%store tokens_bigrams_Corpus_LDA_AE_clean

**DICTIONNAIRE**

In [ ]:
dictionary_AE_2= Dictionary(tokens_bigrams_Corpus_LDA_AE_clean)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE_2))

In [ ]:
# Choix des filtres en fonction de la discussion plus haut
filtre_above_optim = 0.3
filtre_below_optim = 3

dictionary_AE_2.filter_extremes(no_below=filtre_below_optim, no_above=filtre_above_optim)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE_2))

In [ ]:
%store dictionary_AE_2

**LEMMATISATION**

In [ ]:
tokens_bigrams_Corpus_LDA_AE_clean_lemma = tokens_bigrams_Corpus_LDA_AE_lemma

In [ ]:
for index_element in tqdm_notebook(range(len(tokens_bigrams_Corpus_LDA_AE_lemma))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_AE_lemma[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_AE_clean_lemma[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_AE_clean_lemma[index_element]]

**DICTIONNAIRE**

In [ ]:
dictionary_AE_2_lemma= Dictionary(tokens_bigrams_Corpus_LDA_AE_clean_lemma)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE_2_lemma))

In [ ]:
# Choix des filtres en fonction de la discussion plus haut
filtre_above_optim = 0.3
filtre_below_optim = 3

dictionary_AE_2_lemma.filter_extremes(no_below=filtre_below_optim, no_above=filtre_above_optim)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_AE_2_lemma))

In [ ]:
%store tokens_bigrams_Corpus_LDA_AE_clean_lemma
%store dictionary_AE_2_lemma

In [ ]:
def count_tot(tokens_corpus, dictionary):
    "Retourne le nombre total de tokens dans le corpus"
    count_tot = 0
    for document in tqdm_notebook(range(len(tokens_corpus))):
        for word in dictionary:
            count_tot += tokens_corpus[document].count(dictionary[word])
    return count_tot

In [ ]:
# test sur tous les documents, avec le dictionnaire non filtré
# Running time = 1h20
count_tot_base_AE_lemma = count_tot(tokens_corpus = tokens_bigrams_Corpus_LDA_AE_clean_lemma, dictionary = dictionary_AE_2_lemma)
count_tot_base_AE_lemma

In [ ]:
%store count_tot_base_AE_lemma

--------------

**RESUME APRES FILTRE SANS LEMMATISATION**
- Le vocabulaire contient 36100 tokens uniques (différents)
- Le nombre total de tokens est de 4 407 745

-----------------

**RESUME APRES FILTRE APRES LEMMATISATION**
- Le vocabulaire contient 32 374 tokens uniques (différents)

-----------------

### **ii TRAITEMENT POUR ETUDES INTERNATIONALES**

**SANS LEMMATISATION**

In [ ]:
# Création du dictionnaire initial
dictionary_EI = Dictionary(tokens_bigrams_Corpus_LDA_EI)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_EI))

**AVEC LEMMATISATION**

In [ ]:
# Création du dictionnaire
dictionary_EI_lemma = Dictionary(tokens_bigrams_Corpus_LDA_EI_lemma)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_EI_lemma))

**MOTS RARES ET FREQUENTS**

**a) Diversité mots**

**FILTRE ABOVE**

In [ ]:
# Création des dictionnaires selon chaque filtre (à faire une fois pour les 2 stats)
filtres_above = list(range(5,105,5))
dicos_filtres_above_EI = []
for filtre in tqdm(filtres_above):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_EI)
    dico.filter_extremes(no_above=filtre/100)
    dicos_filtres_above_EI.append(dico)

**FILTRE BELOW**

In [ ]:
filtres_below = list(range(1,20)) # filtre below est en nombre absolu de documents
dicos_filtres_below_EI = []
for filtre in tqdm(filtres_below):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_EI)
    dico.filter_extremes(no_below=filtre)
    dicos_filtres_below_EI.append(dico)

**PLOTS**

In [ ]:
def plot_filtre (initiales, relatif=True,type_filtre="above"):
    fig = plt.figure(figsize=(10,5))
    
    if type_filtre == "above":
        filtres = filtres_above
        nb_mots_restants= [len(dico) for dico in dicos_filtres_above_EI]
        plt.xlabel('Seuil de filtre des mots fréquents (en pourcentage d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[-1] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.97)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres, nb_mots_restants, color='k')
    
    elif type_filtre == "below": 
        filtres = filtres_below
        nb_mots_restants= [len(dico) for dico in dicos_filtres_below_EI]
        plt.xlabel('Seuil de filtre des mots rares (en nombre d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[0] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.46)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres, nb_mots_restants, color='burlywood')
    
    plt.ylabel('Taille relative du vocabulaire',fontsize=20)
    #plt.title("Diversité du vocabulaire en fonction du filtre des mots appaissant dans plus d'un certain pourcent de documents")
    plt.title(initiales, fontsize=40)
    plt.savefig("Plots/Filtre vocabulaire/filtre_" + type_filtre+"_absolu_"+initiales+".png")
    plt.show()

In [ ]:
plot_filtre (initiales= 'EI', relatif=True,type_filtre="below")

**b) ALGORITHME DE NORVIG ET CORRECTEUR ORTHOGRAPHIQUE**

Idée : utiliser distance de Levenshtein pour trouver les mots du vocabulaire les plus proches d'un mot donné, et donner la correction la plus probable du mot en fonction de sa fréquence générale dans la langue.

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='fr', distance=1)

**SANS LEMMATISATION**

In [ ]:
tokens_filtré_EI = [dictionary_EI[w] for w in dictionary_EI]

In [ ]:
misspelled = spell.unknown(tokens_filtré_EI)
corrections = []
nb_bigrams = 0
for token in misspelled:
    if '_' in token: # on ne corrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_EI)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_EI)*100,1), "% des tokens.")

In [ ]:
dico_correction = {}
for element in corrections:
    dico_correction[element[0]]= element[1]

**AVEC LEMMATISATION**

In [ ]:
tokens_filtré_EI_lemma = [dictionary_EI_lemma[w] for w in dictionary_EI_lemma]

In [ ]:
misspelled_lemma = spell.unknown(tokens_filtré_EI_lemma)
corrections = []
nb_bigrams = 0
for token in misspelled_lemma:
    if '_' in token: # on ne c_lemmaorrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_EI_lemma)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_EI_lemma)*100,1), "% des tokens.")

In [ ]:
dico_correction = {}
for element in corrections:
    dico_correction[element[0]]= element[1]

**3) FORMATION DU DICTIONNAIRE FINAL**

**SANS LEMMATISATION**

In [ ]:
tokens_bigrams_Corpus_LDA_EI_clean = tokens_bigrams_Corpus_LDA_EI

In [ ]:
for index_element in tqdm_notebook(range(len(tokens_bigrams_Corpus_LDA_EI))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_EI[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_EI_clean[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_EI_clean[index_element]]

In [ ]:
%store tokens_bigrams_Corpus_LDA_AE_clean_lemma
%store dictionary_AE_2_lemma

**AVEC LEMMATISATION**

**i.FILTRE CORRECTION**

In [ ]:
tokens_bigrams_Corpus_LDA_EI_clean_lemma = tokens_bigrams_Corpus_LDA_EI_lemma

In [ ]:
for index_element in tqdm_notebook(range(len(tokens_bigrams_Corpus_LDA_EI_lemma))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_EI_lemma[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_EI_clean_lemma[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_EI_clean_lemma[index_element]]

**ii. DICTIONNAIRE**

In [ ]:
dictionary_EI_2_lemma= Dictionary(tokens_bigrams_Corpus_LDA_EI_clean_lemma)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_EI_2_lemma))

In [ ]:
# Choix des filtres en fonction de la discussion plus haut
filtre_above_optim = 0.3
filtre_below_optim = 3

dictionary_EI_2_lemma.filter_extremes(no_below=filtre_below_optim, no_above=filtre_above_optim)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_EI_2_lemma))

In [ ]:
%store dictionary_EI_2_lemma
%store tokens_bigrams_Corpus_LDA_EI_clean_lemma

**b) Nombre total de tokens dans le corpus**

In [ ]:
%store -r tokens_bigrams_Corpus_LDA_EI_clean
%store -r dictionary_EI_2
from tqdm import tqdm_notebook

In [ ]:
def count_tot(tokens_corpus, dictionary):
    "Retourne le nombre total de tokens dans le corpus"
    count_tot = 0
    for document in tqdm_notebook(range(len(tokens_corpus))):
        for word in dictionary:
            count_tot += tokens_corpus[document].count(dictionary[word])
    return count_tot

In [ ]:
# test sur tous les documents, avec le dictionnaire filtré
# Running time = 1h
count_tot_base_EI = count_tot(tokens_corpus = tokens_bigrams_Corpus_LDA_EI_clean, dictionary = dictionary_EI_2)
count_tot_base_EI

--------------

**RESUME APRES FILTRE SANS LEMMATISATION**
- Le vocabulaire contient 41.585 tokens uniques
- Le nombre total de tokens est de 2 306 993
- Le nombre total d'articles est de 1094

-----------------

**RESUME APRES FILTRE AVEC LEMMATISATION**
- Le vocabulaire contient 37.343 tokens uniques
- Le nombre total de tokens est de 
- Le nombre total d'articles est de 1094

-----------------

In [ ]:
%store count_tot_base_EI

### **iii. TRAITEMENT POUR RELATIONS INDUSTRIELLES**

**SANS LEMMATISATION**

In [ ]:
# Création du dictionnaire initial
dictionary_RI = Dictionary(tokens_bigrams_Corpus_LDA_RI)
print('Nombre de tokens uniques dans les documents avant pré-processing:', len(dictionary_RI))

**AVEC LEMMATISATION**

In [ ]:
# Création du dictionnaire
dictionary_RI_lemma = Dictionary(tokens_bigrams_Corpus_LDA_RI_lemma)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_RI_lemma))

**MOTS RARES ET FREQUENTS**

**a) Diversité mots**

**FILTRE ABOVE**

In [ ]:
# Création des dictionnaires selon chaque filtre (à faire une fois pour les 2 stats)
filtres_above = list(range(5,105,5))
dicos_filtres_above_RI = []
for filtre in tqdm(filtres_above):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_RI)
    dico.filter_extremes(no_above=filtre/100)
    dicos_filtres_above_RI.append(dico)

**FILTRE BELOW**

In [ ]:
filtres_below = list(range(1,20)) # filtre below est en nombre absolu de documents
dicos_filtres_below_RI = []
for filtre in tqdm(filtres_below):
    dico = Dictionary(tokens_bigrams_Corpus_LDA_RI)
    dico.filter_extremes(no_below=filtre)
    dicos_filtres_below_RI.append(dico)

**PLOTS**

In [ ]:
def plot_filtre (initiales, relatif=True,type_filtre="above"):
    fig = plt.figure(figsize=(10,5))
    
    if type_filtre == "above":
        filtres = filtres_above
        nb_mots_restants= [len(dico) for dico in dicos_filtres_above_RI]
        plt.xlabel('Seuil de filtre des mots fréquents (en pourcentage d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[-1] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.98)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres, nb_mots_restants, color='k')
    
    elif type_filtre == "below": 
        filtres = filtres_below
        nb_mots_restants= [len(dico) for dico in dicos_filtres_below_RI]
        plt.xlabel('Seuil de filtre des mots rares (en nombre d\'articles)',fontsize=20)
        if relatif: 
            nb_mots_restants = [element/nb_mots_restants[0] for element in nb_mots_restants]
            plt.axhline(color = 'r', y=0.45)
        else : 
            plt.axhline(color = 'r', y=30000)
        plt.bar(filtres, nb_mots_restants, color='burlywood')
    
    plt.ylabel('Taille relative du vocabulaire',fontsize=20)
    #plt.title("Diversité du vocabulaire en fonction du filtre des mots appaissant dans plus d'un certain pourcent de documents")
    plt.title(initiales, fontsize=40)
    plt.savefig("Plots/Filtre vocabulaire/filtre_" + type_filtre+"_absolu_"+initiales+".png")
    plt.show()

In [ ]:
plot_filtre (initiales= 'RI', relatif=True,type_filtre="below")

In [ ]:
plot_filtre (initiales= 'RI', relatif=True,type_filtre="below")

**2) ALGORITHME DE NORVIG ET CORRECTEUR ORTHOGRAPHIQUE**

Idée : utiliser distance de Levenshtein pour trouver les mots du vocabulaire les plus proches d'un mot donné, et donner la correction la plus probable du mot en fonction de sa fréquence générale dans la langue.

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(language='fr', distance=1)

**SANS LEMMATISATION**

In [ ]:
tokens_filtré_RI = [dictionary_RI[w] for w in dictionary_RI]

In [ ]:
misspelled = spell.unknown(tokens_filtré_RI)
corrections = []
nb_bigrams = 0
for token in misspelled:
    if '_' in token: # on ne corrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_RI)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_RI)*100,1), "% des tokens.")

In [ ]:
dico_correction = {}
for element in corrections:
    dico_correction[element[0]]= element[1]

**AVEC LEMMATISATION**

In [ ]:
tokens_filtré_RI_lemma = [dictionary_RI_lemma[w] for w in dictionary_RI_lemma]

In [ ]:
misspelled_lemma = spell.unknown(tokens_filtré_RI_lemma)
corrections = []
nb_bigrams = 0
for token in misspelled_lemma:
    if '_' in token: # on ne corrige pas les n_grams mais on les compte
         nb_bigrams +=1
    else: # on trouve la correction la plus probable pour le token courant
        corrections.append((token, spell.correction(token)))

In [ ]:
print("", round(len(corrections)/len(tokens_filtré_RI_lemma)*100, 1), "% des mots ont été corrigés.\n",
      "Il y a", nb_bigrams, "bigrammes dans le dictionnaire filtré ce qui représente ", round(nb_bigrams/len(dictionary_RI_lemma)*100,1), "% des tokens.")

In [ ]:
dico_correction = {}
for element in corrections:
    dico_correction[element[0]]= element[1]

**3) FORMATION DU DICTIONNAIRE FINAL**

**SANS LEMMATISATION**

**i. FILTRE CORRECTION**

In [ ]:
tokens_bigrams_Corpus_LDA_RI_clean = tokens_bigrams_Corpus_LDA_RI

In [ ]:
for index_element in tqdm_notebook(range(len(tokens_bigrams_Corpus_LDA_RI))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_RI[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_RI_clean[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_RI_clean[index_element]]

**ii.DICTIONNAIRE**

In [ ]:
dictionary_RI_2= Dictionary(tokens_bigrams_Corpus_LDA_RI_clean)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_RI_2))

In [ ]:
# Choix des filtres en fonction de la discussion plus haut
filtre_above_optim = 0.3
filtre_below_optim = 3

dictionary_RI_2.filter_extremes(no_below=filtre_below_optim, no_above=filtre_above_optim)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_RI_2))

In [ ]:
%store dictionary_RI_2
%store tokens_bigrams_Corpus_LDA_RI_clean

**AVEC LEMMATISATION**

**i.FILTRE CORRECTION**

In [ ]:
tokens_bigrams_Corpus_LDA_RI_clean_lemma = tokens_bigrams_Corpus_LDA_RI_lemma

In [ ]:
for index_element in tqdm_notebook(range(len(tokens_bigrams_Corpus_LDA_RI_lemma))): # parcours de chaque liste de token
    for token in tokens_bigrams_Corpus_LDA_RI_lemma[index_element]: # évaluation de chaque token 
        if token in dico_correction: # si le token a été corrigé, le remplacer par sa correction
            tokens_bigrams_Corpus_LDA_RI_clean_lemma[index_element] = [dico_correction[token] if x == token else x for x in tokens_bigrams_Corpus_LDA_RI_clean_lemma[index_element]]

**ii. DICTIONNAIRE**

In [ ]:
dictionary_RI_2_lemma= Dictionary(tokens_bigrams_Corpus_LDA_RI_clean_lemma)
print('Nombre de tokens uniques dans les documents avant pré-processing:', len(dictionary_RI_2_lemma))

In [ ]:
# Choix des filtres en fonction de la discussion plus haut
filtre_above_optim = 0.3
filtre_below_optim = 3

dictionary_RI_2_lemma.filter_extremes(no_below=filtre_below_optim, no_above=filtre_above_optim)
print('Nombre de tokens uniques dans les documents après pré-processing:', len(dictionary_RI_2_lemma))

In [ ]:
%store dictionary_RI_2_lemma
%store tokens_bigrams_Corpus_LDA_RI_clean_lemma

----------------------

**b) Nombre total de tokens dans le corpus**

In [ ]:
%store -r tokens_bigrams_Corpus_LDA_RI_clean_lemma
%store -r dictionary_RI_2_lemma
from tqdm import tqdm_notebook

In [ ]:
def count_tot(tokens_corpus, dictionary):
    "Retourne le nombre total de tokens dans le corpus"
    count_tot = 0
    for document in tqdm_notebook(range(len(tokens_corpus))):
        for word in dictionary:
            count_tot += tokens_corpus[document].count(dictionary[word])
    return count_tot

In [ ]:
# test sur tous les documents, avec le dictionnaire filtré
# Running time = 1h
count_tot_base_RI = count_tot(tokens_corpus = tokens_bigrams_Corpus_LDA_RI_clean, dictionary = dictionary_RI_2)
count_tot_base_RI

In [ ]:
%store count_tot_base_RI

--------------

**RESUME APRES FILTRE SANS LEMMATISATION**
- Le vocabulaire contient 39205
- Le nombre total de tokens est de 2486175


--------------

**RESUME APRES FILTRE APRES LEMMATISATION**
- Le vocabulaire contient 97 333
- Le nombre total de tokens est de 

-----------------